# Disease Information App Tutorial

### Building an Interactive Browser-Based Q&A System using the Perplexity (Sonar) API

In this tutorial, you'll learn how to build a disease Q&A system that:

- Queries the Perplexity API with disease-related questions
- Generates a standalone HTML interface with interactive elements
- Displays results in both the notebook and a web browser

Let's get started!

## Overview

In this notebook, you will:

- **Set up your environment:** Install required packages and import dependencies.
- **Configure the API:** Set your Perplexity (Sonar) API key and endpoint.
- **Define functions:** Create functions to query the API, generate an HTML UI, display results, and launch the browser UI.
- **Test the app:** Run examples to test the API and view the interactive interface.

Follow the steps below to build your interactive Disease Information App.

## Prerequisites

Make sure you have the following installed:

- Python 3.x
- The following Python packages: `requests`, `pandas`, and `IPython`

You can install these packages using pip:

```bash
pip install requests pandas jupyterlab
```

Also, replace the placeholder API key (`API_KEY`) with your actual Perplexity API key.

In [ ]:
# Step 1: Setup and Dependencies

import requests
import json
import pandas as pd
from IPython.display import HTML, display, IFrame
import os
import webbrowser
from pathlib import Path

print('Setup complete.')

In [ ]:
# Step 2: API Configuration

# Replace with your actual Perplexity API key
API_KEY = 'YOUR_ACTUAL_API_KEY'
API_ENDPOINT = 'https://api.perplexity.ai/chat/completions'

print('API configuration set.')

In [ ]:
# Step 3: Create the API Query Function

def ask_disease_question(question):
    """
    Send a disease-related question to the Perplexity API and parse the response.
    
    Args:
        question (str): The question about a disease.
        
    Returns:
        dict: JSON response with keys 'overview', 'causes', 'treatments', and 'citations'.
    """
    prompt = f"""
You are a medical assistant. Please answer the following question about a disease and provide only valid JSON output.
The JSON object must have exactly four keys: "overview", "causes", "treatments", and "citations".
For example:
{
  "overview": "A brief description of the disease.",
  "causes": "The causes of the disease.",
  "treatments": "Possible treatments for the disease.",
  "citations": ["https://example.com/citation1", "https://example.com/citation2"]
}
Now answer this question:
"{question}"
    """.strip()

    payload = {
        "model": "sonar-pro",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    try:
        headers = {
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        }
        response = requests.post(API_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()

        result = response.json()
        
        if result.get("choices") and len(result["choices"]) > 0:
            content = result["choices"][0]["message"]["content"]
            try:
                return json.loads(content)
            except json.JSONDecodeError:
                print("Failed to parse JSON output from API. Raw output:")
                print(content)
                return None
        else:
            print("No answer provided in the response.")
            return None

    except Exception as e:
        print(f"Error: {e}")
        return None

print('ask_disease_question function defined.')

In [ ]:
# Step 4: Create the HTML User Interface File

def create_html_ui(api_key, output_path="disease_qa.html"):
    """
    Create an HTML file with the disease Q&A interface.
    
    Args:
        api_key (str): The Perplexity API key.
        output_path (str): Path where the HTML file will be saved.
        
    Returns:
        str: The absolute path to the created HTML file.
    """
    html_content = f"""<!DOCTYPE html>
<html lang=\"en\">
<head>
  <meta charset=\"UTF-8\" />
  <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />
  <title>Disease Q&A Knowledge Card</title>
  <style>
    /* Base styling */
    body {{
      font-family: Helvetica, Arial, sans-serif;
      background-color: #F7F7F8;
      color: #333;
      margin: 0;
      padding: 0;
    }}
    .container {{
      max-width: 800px;
      margin: 2rem auto;
      padding: 1rem;
    }}
    h1 {{
      text-align: center;
      color: #111;
      margin-bottom: 1rem;
    }}
    /* Form styling */
    #qaForm {{
      display: flex;
      justify-content: center;
      margin-bottom: 1.5rem;
    }}
    #question {{
      flex: 1;
      padding: 0.75rem;
      font-size: 1.2rem;
      border: 1px solid #ccc;
      border-radius: 4px;
      margin-right: 0.5rem;
    }}
    #askButton {{
      padding: 0.75rem 1rem;
      font-size: 1.2rem;
      background-color: #10a37f;
      border: none;
      color: #fff;
      border-radius: 4px;
      cursor: pointer;
      transition: background-color 0.2s ease;
    }}
    #askButton:hover {{
      background-color: #0d8a66;
    }}
    /* Knowledge card and citations styling */
    #knowledgeCard, #citationsCard {{
      background: #fff;
      border: 1px solid #e0e0e0;
      border-radius: 8px;
      padding: 1rem;
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
      margin-top: 1.5rem;
      display: none;
    }}
    table {{
      width: 100%;
      border-collapse: collapse;
    }}
    th, td {{
      text-align: left;
      padding: 0.75rem;
      border-bottom: 1px solid #e0e0e0;
    }}
    th {{
      background-color: #fafafa;
      width: 25%;
    }}
    /* Citations list styling */
    #citationsList {{
      list-style-type: disc;
      padding-left: 20px;
    }}
    #citationsList li {{
      margin-bottom: 0.5rem;
    }}
    #citationsList a {{
      color: #10a37f;
      text-decoration: none;
    }}
    #citationsList a:hover {{
      text-decoration: underline;
    }}
    /* Loading overlay styling */
    #loadingOverlay {{
      position: fixed;
      top: 0;
      left: 0;
      width: 100%;
      height: 100%;
      background: rgba(255, 255, 255, 0.8);
      display: flex;
      justify-content: center;
      align-items: center;
      z-index: 9999;
      display: none;
    }}
    .spinner {{
      border: 8px solid #f3f3f3;
      border-top: 8px solid #10a37f;
      border-radius: 50%;
      width: 60px;
      height: 60px;
      animation: spin 1s linear infinite;
    }}
    @keyframes spin {{
      0% {{ transform: rotate(0deg); }}
      100% {{ transform: rotate(360deg); }}
    }}
    .footer {{
      text-align: center;
      margin-top: 2rem;
      padding: 1rem;
      font-size: 0.9rem;
      color: #777;
    }}
  </style>
</head>
<body>
  <!-- Loading Overlay -->
  <div id=\"loadingOverlay\">
    <div class=\"spinner\"></div>
  </div>

  <div class=\"container\">
    <h1>Disease Q&A</h1>
    <form id=\"qaForm\">
      <input type=\"text\" id=\"question\" placeholder=\"Ask a question about a disease (e.g., 'What is stroke?')\" required>
      <button type=\"submit\" id=\"askButton\">Ask</button>
    </form>

    <!-- Knowledge card container -->
    <div id=\"knowledgeCard\">
      <h2>Knowledge Card</h2>
      <table>
        <tr>
          <th>Overview</th>
          <td id=\"overview\"></td>
        </tr>
        <tr>
          <th>Causes</th>
          <td id=\"causes\"></td>
        </tr>
        <tr>
          <th>Treatments</th>
          <td id=\"treatments\"></td>
        </tr>
      </table>
    </div>

    <!-- Citations container -->
    <div id=\"citationsCard\">
      <h2>Citations</h2>
      <ul id=\"citationsList\"></ul>
    </div>
    
    <div class=\"footer\">
      <p>Created with Sonar API</p>
    </div>
  </div>

  <script>
    const API_KEY = '{api_key}';
    const API_ENDPOINT = 'https://api.perplexity.ai/chat/completions';

    async function askDiseaseQuestion(question) {
      const prompt = `
You are a medical assistant. Please answer the following question about a disease and provide only valid JSON output.
The JSON object must have exactly four keys: "overview", "causes", "treatments", and "citations".
For example:
{
  "overview": "A brief description of the disease.",
  "causes": "The causes of the disease.",
  "treatments": "Possible treatments for the disease.",
  "citations": ["https://example.com/citation1", "https://example.com/citation2"]
}
Now answer this question:
${question}`.trim();

      const payload = {
        model: 'sonar-pro',
        messages: [
          { role: 'user', content: prompt }
        ]
      };

      try {
        const response = await fetch(API_ENDPOINT, {
          method: 'POST',
          headers: {
            'Authorization': `Bearer ${API_KEY}`,
            'Content-Type': 'application/json'
          },
          body: JSON.stringify(payload)
        });

        if (!response.ok) {
          throw new Error(`Error: ${response.status} - ${response.statusText}`);
        }

        const result = await response.json();

        if (result.choices && result.choices.length > 0) {
          const content = result.choices[0].message.content;
          try {
            const structuredOutput = JSON.parse(content);
            return structuredOutput;
          } catch (jsonErr) {
            throw new Error('Failed to parse JSON output from API. Raw output: ' + content);
          }
        } else {
          throw new Error('No answer provided in the response.');
        }
      } catch (error) {
        console.error(error);
        alert(error);
      }
    }

    function setLoading(isLoading) {
      document.getElementById('loadingOverlay').style.display = isLoading ? 'flex' : 'none';
    }

    document.getElementById('qaForm').addEventListener('submit', async (event) => {
      event.preventDefault();
      document.getElementById('knowledgeCard').style.display = 'none';
      document.getElementById('citationsCard').style.display = 'none';
      setLoading(true);
      const question = document.getElementById('question').value;
      const data = await askDiseaseQuestion(question);
      setLoading(false);

      if (data) {
        document.getElementById('overview').textContent = data.overview || 'N/A';
        document.getElementById('causes').textContent = data.causes || 'N/A';
        document.getElementById('treatments').textContent = data.treatments || 'N/A';
        document.getElementById('knowledgeCard').style.display = 'block';

        const citationsList = document.getElementById('citationsList');
        citationsList.innerHTML = '';
        if (Array.isArray(data.citations) && data.citations.length > 0) {
          data.citations.forEach(citation => {
            const li = document.createElement('li');
            const link = document.createElement('a');
            link.href = citation;
            link.textContent = citation;
            link.target = '_blank';
            li.appendChild(link);
            citationsList.appendChild(li);
          });
        } else {
          const li = document.createElement('li');
          li.textContent = 'No citations provided.';
          citationsList.appendChild(li);
        }
        document.getElementById('citationsCard').style.display = 'block';
      }
    });
  </script>
</body>
</html>
"""

    with open(output_path, 'w') as f:
        f.write(html_content)
    full_path = os.path.abspath(output_path)
    return full_path

print('create_html_ui function defined.')

In [ ]:
# Step 5: Display API Results in the Notebook

def display_results(data):
    """
    Display the API results in a structured format using a pandas DataFrame and text output.
    
    Args:
        data (dict): The parsed JSON data from the API.
    """
    if not data:
        print("No data to display.")
        return
    
    df = pd.DataFrame({
        "Category": ["Overview", "Causes", "Treatments"],
        "Information": [
            data.get("overview", "N/A"),
            data.get("causes", "N/A"),
            data.get("treatments", "N/A")
        ]
    })
    
    print("\n💡 Knowledge Card:")
    display(df.style.set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#fafafa'), ('color', '#333'), ('font-weight', 'bold')]},
        {'selector': 'td', 'props': [('padding', '10px')]},
    ]))
    
    print("\n📚 Citations:")
    if data.get("citations") and isinstance(data["citations"], list) and len(data["citations"]) > 0:
        for i, citation in enumerate(data["citations"], 1):
            print(f"{i}. {citation}")
    else:
        print("No citations provided.")

print('display_results function defined.')

In [ ]:
# Step 6: Launch the Browser UI

def launch_browser_ui(api_key=API_KEY, html_path="disease_qa.html"):
    """
    Generate and open the HTML UI in a web browser.
    
    Args:
        api_key (str): The Perplexity API key.
        html_path (str): Path to save the HTML file.
        
    Returns:
        str: The absolute path to the created HTML file.
    """
    full_path = create_html_ui(api_key, html_path)
    file_url = f"file://{full_path}"
    print(f"Opening browser UI: {file_url}")
    webbrowser.open(file_url)
    return full_path

print('launch_browser_ui function defined.')

In [ ]:
# Step 7: Example Usage

def test_api_in_notebook():
    print("Example 1: Direct API Call")
    print("-------------------------")
    example_question = "What is diabetes?"
    print(f"Question: {example_question}")
    print("Sending request to Perplexity API...")
    # Uncomment the following lines to make an actual API call:
    # result = ask_disease_question(example_question)
    # display_results(result)
    print("(API call commented out to avoid using your API quota)")
    print("\n")

def launch_browser_app():
    print("Example 2: Launching Browser UI")
    print("-----------------------------")
    print("Generating HTML file and opening in browser...")
    path = launch_browser_ui()
    print(f"\nHTML file created at: {path}")
    print("\nIf the browser doesn't open automatically, you can manually open the file above.")
    try:
        display(HTML(f'<p>Preview of UI (may not work in all environments):</p>'))
        display(IFrame(path, width='100%', height=600))
    except Exception as e:
        print("Preview not available in this environment.")

# Run the examples
test_api_in_notebook()
launch_browser_app()

print('Example usage executed.')

## Tutorial Explanation & Extensions

### How This Tutorial Works:

1. **Backend (Python):**
   - Generates an HTML file with an interactive UI.
   - Contains functions to query the Perplexity API and display structured results.

2. **Frontend (HTML/JavaScript):**
   - Provides a user-friendly interface for entering disease-related questions.
   - Makes API calls directly from the browser and updates the UI dynamically.

### Possible Extensions:

- **Security:** Use a backend server (e.g., Flask) to securely manage your API key.
- **Caching:** Implement caching to reduce redundant API calls.
- **History:** Add a feature to view previous questions and answers.
- **UI Enhancements:** Customize the HTML/CSS for a better appearance.
- **Additional Data:** Expand the API prompt to include more information or data visualizations.


## Conclusion

This tutorial demonstrated how to create a browser-based Disease Q&A system using the Perplexity (Sonar) API. 

- **Interactive UI:** Generate a standalone HTML file with embedded CSS and JavaScript.
- **API Integration:** Query the Perplexity API to fetch structured responses about diseases.
- **Display Options:** View results directly in the notebook or launch them in your browser.

Remember to replace the API key placeholder with your own API key before running the app.

Happy coding!